# Caminhos Mínimos

**\> Se aplica a:**
- Grafos direcionados

**\> Queremos encontrar:**
- Distância de cada vértice até a um dado vértice source (s).

## Estruturas de dados a serem usadas

In [104]:
"""Same as before however added method for listing edges"""


class Node:  
    def __init__(self, name, color=None, d=None, pi=None):
        self.name = name
        self.adj_list = []
        self.color = color  # estado: visitado, não visitado, explorado
        self.d = d  # distancia do vértice origem
        self.pi = pi  # vértice predecessor
   
    # def __repr__(self):
    #     return repr(self.name)
    
class Graph:
    def __init__(self, num_vertices=None, undirected=True):
        self.num_vertices = num_vertices
        self.nodes = [Node(v) for v in range(num_vertices)]
        self.undirected = undirected

    def __getitem__(self, item):
        return self.nodes[item]
    
    def add_edge(self, source, dest, weight):
        self.nodes[source].adj_list.append((self.nodes[dest], weight))
        if self.undirected:
            self.nodes[dest].adj_list.append((self.nodes[source], weight))
        
    def print(self):
        for node in self.nodes:
            print(f"Node {node.name} -> {[(n.name, w) for n, w in node.adj_list]}")
            
    def get_list_edges(self):
        edges = []
        for u in self.nodes:
            for v, w in u.adj_list:
                if self.undirected:  # So it won't repeat edges
                    first = min(u.name, v.name)
                    second = v.name if first == u.name else u.name
                    edge = (first, second, w)
                    edges.append(edge) if edge not in edges else None
                else:
                    edges.append((u.name, v.name, w))

        return edges
    
    def get_sorted_list_edges(self):
        edges = self.get_list_edges()
        return sorted(edges, key=lambda tup: tup[2])

    def find_edge_weight(self, e1, e2):
        for v, w in self.nodes[e1].adj_list:
            if v.name == e2:
                return w
        return 'Not found'

        

## Belman Ford
- Para cada vértice, pego sua distância até a source.
- Também indica se tem ciclos.
- Aceita pesos negativos.

### Inicialização do grafo

In [9]:
g = Graph(5)
g.add_edge(0, 1, 3)
g.add_edge(0, 3, 7)
g.add_edge(0, 4, 8)
g.add_edge(1, 2, 1)
g.add_edge(1, 3, 4)
g.add_edge(2, 3, 2)
g.add_edge(3, 4, 3)
g.print()

Node 0 -> [(1, 3), (3, 7), (4, 8)]
Node 1 -> [(0, 3), (2, 1), (3, 4)]
Node 2 -> [(1, 1), (3, 2)]
Node 3 -> [(0, 7), (1, 4), (2, 2), (4, 3)]
Node 4 -> [(0, 8), (3, 3)]


### Algorítmo

In [10]:
INF = 99999999999

class BellmanFord():
    def __init__(self, G):
        self.G = G
        self.E = G.get_list_edges()
        
    def initialize_single_source(self, s):
        """Inicialização dos nós com distancias infinitas até a 
        source e predecessores nulos"""
        
        for v in self.G.nodes:
            v.d = INF
            v.pi = None
            
        self.G[s].d = 0

    def relax(self, u, v, w):
        """Verifica se o o caminho para o vertice `u` melhora 
        o caminho minimo encontrado até agora para o vértice `v`.
        Se sim, atualiza o valor de `v.d` com o peso até agora, 
        e adiciona `u` como seu predecessor."""
        
        if self.G[v].d > self.G[u].d + w:
            self.G[v].d = self.G[u].d + w
            self.G[v].pi = self.G[u]
        
    def bellman_ford(self, s):
        self.initialize_single_source(s)
        
        # "Relaxa" as arestas
        for i in range(len(self.G.nodes) - 1):
            for u, v, w in self.E:
                self.relax(u, v, w)
        
        # Printando
        [print(f"Vertex: {n.name}, shortest_discance_from_source: {n.d}, pred: {n.pi}") for n in self.G.nodes]
        
        # Verifica se tem ciclos negativos
        for edge in self.E:
            w = edge[2]
            if self.G[v].d > self.G[u].d + w:
                return False
        return True

In [11]:
g = Graph(5)
g.add_edge(0, 1, 3)
g.add_edge(0, 3, 7)
g.add_edge(0, 4, 8)
g.add_edge(1, 2, 1)
g.add_edge(1, 3, 4)
g.add_edge(2, 3, 2)
g.add_edge(3, 4, 3)
g.print()

Node 0 -> [(1, 3), (3, 7), (4, 8)]
Node 1 -> [(0, 3), (2, 1), (3, 4)]
Node 2 -> [(1, 1), (3, 2)]
Node 3 -> [(0, 7), (1, 4), (2, 2), (4, 3)]
Node 4 -> [(0, 8), (3, 3)]


In [12]:
bf = BellmanFord(g)

bf.bellman_ford(0)

Vertex: 0, shortest_discance_from_source: 0, pred: None
Vertex: 1, shortest_discance_from_source: 3, pred: <__main__.Node object at 0x000001FF86586310>
Vertex: 2, shortest_discance_from_source: 4, pred: <__main__.Node object at 0x000001FF8616FDF0>
Vertex: 3, shortest_discance_from_source: 6, pred: <__main__.Node object at 0x000001FF87B2FB50>
Vertex: 4, shortest_discance_from_source: 8, pred: <__main__.Node object at 0x000001FF86586310>


False

## Dijkstra
- Para cada vértice, pego sua distância até a source.
- Também indica se tem ciclos.
- Aceita pesos negativos.

### Inicialização do grafo

In [1]:
"""Same as before however added method for listing edges"""


class Node:  
    def __init__(self, name, color=None, d=None, pi=None):
        self.name = name
        self.adj_list = []
        self.color = color  # estado: visitado, não visitado, explorado
        self.d = d  # distancia do vértice origem
        self.pi = pi  # vértice predecessor
   
    def __repr__(self):
        return repr(self.name)
    
class Graph:
    def __init__(self, num_vertices=None, undirected=True):
        self.num_vertices = num_vertices
        self.nodes = [Node(v) for v in range(num_vertices)]
        self.undirected = undirected

    def __getitem__(self, item):
        return self.nodes[item]
    
    def add_edge(self, source, dest, weight):
        self.nodes[source].adj_list.append((self.nodes[dest], weight))
        if self.undirected:
            self.nodes[dest].adj_list.append((self.nodes[source], weight))
        
    def print(self):
        for node in self.nodes:
            print(f"Node {node.name} -> {[(n.name, w) for n, w in node.adj_list]}")
            
    def get_list_edges(self):
        edges = []
        for u in self.nodes:
            for v, w in u.adj_list:
                if self.undirected:  # So it won't repeat edges
                    first = min(u.name, v.name)
                    second = v.name if first == u.name else u.name
                    edge = (first, second, w)
                    edges.append(edge) if edge not in edges else None
                else:
                    edges.append((u.name, v.name, w))

        return edges
    
    def get_sorted_list_edges(self):
        edges = self.get_list_edges()
        return sorted(edges, key=lambda tup: tup[2])

    def find_edge_weight(self, e1, e2):
        for v, w in self.nodes[e1].adj_list:
            if v.name == e2:
                return w
        return 'Not found'

        

### Estrutura de dados de controle

In [2]:
class PriorityQueue(object):
    def __init__(self):
        self.queue = []
  
    def __str__(self):
        return ' '.join([str(i) for i in self.queue])
  
    # for checking if the queue is empty
    def isEmpty(self):
        return len(self.queue) == 0
  
    # for inserting an element in the queue
    def insert(self, data):
        self.queue.append(data)
  
    # for popping an element based on Priority
    def extract_max(self):
        try:
            max_val = 0
            for i in range(len(self.queue)):
                if self.queue[i].d > self.queue[max_val].d:
                    max_val = i
            item = self.queue[max_val]
            del self.queue[max_val]
            return item
        except IndexError:
            print()
            exit()
            
    def extract_min(self):
        try:
            min_val = 0
            for i in range(len(self.queue)):
                if self.queue[i].d < self.queue[min_val].d:
                    min_val = i
            item = self.queue[min_val]
            del self.queue[min_val]
            return item
        except IndexError:
            print()
            exit()
    
    def decrease_key(self, v, d):
        for i, n in enumerate(self.queue):
            if n.name == v.name:
                n.d = d           

### Algoritmo

In [9]:
INF = 99999999999

class Dijkstra():
    def __init__(self, G):
        self.G = G
        self.E = G.get_list_edges()
        
    def initialize_single_source(self, s):
        """Inicialização dos nós com distancias infinitas até a 
        source e predecessores nulos"""
        
        for v in self.G.nodes:
            v.d = INF
            v.pi = None
            
        self.G[s].d = 0

    def relax(self, u, v, w):
        """Verifica se o o caminho para o vertice `u` melhora 
        o caminho minimo encontrado até agora para o vértice `v`.
        Se sim, atualiza o valor de `v.d` com o peso até agora, 
        e adiciona `u` como seu predecessor."""
        
        decreased = False
        if self.G[v].d > self.G[u].d + w:
            decreased = True if self.G[u].d + w < self.G[v].d else False
            self.G[v].d = self.G[u].d + w
            self.G[v].pi = self.G[u]
            
        return decreased
        
    def dijkstra(self, s):
        self.initialize_single_source(s)
        
        S = set()
        Q = PriorityQueue()

        for u in self.G.nodes:
            Q.insert(u)
            
        counter = 0
        while Q.queue != []:
            u = Q.extract_min()
            S = S.union({u})
            for v, w in u.adj_list:
                decreased = self.relax(u.name, v.name, w)
                if decreased:
                    Q.decrease_key(v, v.d)
            

        [print(f"Vertex: {n.name}, shortest_discance_from_source: {n.d}") for n in S]
cal

In [10]:
g = Graph(5)
g.add_edge(0, 1, 3)
g.add_edge(0, 3, 7)
g.add_edge(0, 4, 8)
g.add_edge(1, 2, 1)
g.add_edge(1, 3, 4)
g.add_edge(2, 3, 2)
g.add_edge(3, 4, 3)
g.print()

Node 0 -> [(1, 3), (3, 7), (4, 8)]
Node 1 -> [(0, 3), (2, 1), (3, 4)]
Node 2 -> [(1, 1), (3, 2)]
Node 3 -> [(0, 7), (1, 4), (2, 2), (4, 3)]
Node 4 -> [(0, 8), (3, 3)]


In [5]:
d = Dijkstra(g)
d.dijkstra(0)

Vertex: 1, shortest_discance_from_source: 3
Vertex: 3, shortest_discance_from_source: 6
Vertex: 4, shortest_discance_from_source: 8
Vertex: 2, shortest_discance_from_source: 4
Vertex: 0, shortest_discance_from_source: 0
